In [ ]:
cd 'deep-learning-models/'

In [ ]:
import numpy as np
from resnet50 import ResNet50
from keras.preprocessing import image
from imagenet_utils import preprocess_input, decode_predictions
from keras.models import Model
import scipy.spatial.distance
import time
import json


In [ ]:
def load_image(img_path):
    data = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(data)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [ ]:
#def get_base_model():
#    base_model = ResNet50(weights='imagenet',include_top=False)
#    return base_model

def get_models():
    # include_top needs to be True for this to work
    base_model = ResNet50(weights='imagenet',include_top=True)
    model = Model(input=base_model.input, output=base_model.get_layer('flatten_1').output)
    return (model, base_model)

def image_features(img,model):
    features = np.zeros((1,2048),dtype=np.float16)
    #model = Model(input=base_model.input, output=base_model.get_layer('flatten_1').output)
    predictions = model.predict(img)
    return predictions

In [ ]:
model, base_model = get_models()

In [ ]:
# model = Model(input=bm.input, output=bm.get_layer('flatten_1').output)

In [ ]:
def save_model_workaround(model, model_output_file, weights_output_file):
    print('saving model   to {}'.format(model_output_file))
    print('saving weignts to {}'.format(weights_output_file))
    # serialize model to JSON
    model_json = model.to_json()
    with open(model_output_file, 'w') as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(weights_output_file)


def load_model_workaround(model_output_file, weights_output_file):
    # load json and create model
    json_file = open(model_output_file, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(weights_output_file)
    return loaded_model

In [ ]:
save_model_workaround(model,
                      '/local_data/dgrossman/model_save_dir/dgcars_resenet.model.json',
                      '/local_data/dgrossman/model_save_dir/dgcars_resenet.weights.hdf5')

In [ ]:
# print(scipy.spatial.distance.cosine(feature1,feature2),'good')
# print(scipy.spatial.distance.cosine(feature1,feature3),'bad')
# print(scipy.spatial.distance.cosine(feature2,feature3),'bad')

In [ ]:
prefix = '/local_data/dgrossman/VeRi/'


In [ ]:
tLines = open(prefix + 'trainingLines','r')
trainingList = list()
for tLine in tLines:
    tLine = tLine.strip()
    tLine = tLine.replace('"','')
    parts = tLine.split(' ')
    ldict = dict()
    for part in parts:
        l, r = part.split('=')
        ldict[l]=str(r)
    trainingList.append(ldict)
tLines.close()

In [ ]:
trainingList = list()
tLines = open(prefix + 'test_features.json','r')
for line in tLines:
    line = line.strip()
    line = json.loads(line)
    trainingList.append(line)

In [ ]:
trainingList[0]

In [ ]:
import json
#outFile = open('/local_data/dgrossman/VeRi/training_features.json','w')
outFile = open('/local_data/dgrossman/VeRi/test_features.json','w')

batchSize = 1000
start = time.time()
for idx,line in enumerate(trainingList):
    tempd = dict()
    if idx % batchSize == 0:
        end = time.time() - start
        start = time.time()
        print ('total {0} batch {1} images in {2} seconds'.format(idx,batchSize,end))
    #img = load_image(prefix + 'image_train/'+line['imageName'])
    img = load_image(prefix + 'image_test/'+line['imageName'])
    feature = image_features(img, model) 
    tempd['resnet50'] = feature.tolist()[0]
    tempd.update(line)
    outFile.write(json.dumps(tempd)+'\n')
outFile.close()

In [ ]:
funtime = open('/local_data/dgrossman/VeRi/training_features.json','r')
work = list()
for line in funtime:
    line = line.strip()
    line = json.loads(line)
    work.append(line)
funtime.close()

In [ ]:
work[0]
